In [1]:
! pip install tf_agents[reverb]

^C


In [3]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import environment_simulation as env
from ReplayBuffer import ReplayBuffer
from DDPG import Actor,Critic 
from Agent import Agent
from google.colab import files ##保存图片
import tf_agents
import torch
print(tf.__version__) # 检查是否装好包 2.9.1
print(torch.__version__)

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
import logging
logging.disable(logging.WARNING)


导入程序包，并进行检查

In [ ]:
### 主程序设定
numAPuser = 5
numRU = 8
linkmode = 'uplink'
max_iteration = 5000
np.random.seed(9) 


test_env = env.environment_base(numAPuser,numRU)
userinfo = test_env.senario_4_cells()
channel_gain = test_env.channel_gain()
AP1_RU_mapper = test_env.n_AP_RU_mapper()
AP2_RU_mapper = test_env.n_AP_RU_mapper()
AP3_RU_mapper = test_env.n_AP_RU_mapper()

test_agent = Agent(numAPuser,numRU)

reward_history = []
system_bitrate_history = []
critic_loss_history = []
actor_loss_history = []
user_unconform_history = []
ru_unconform_history = []
critic_value_history = []
target_history = []
ag_history = []
cg_history = []
print(AP3_RU_mapper)


[[0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1.]]


In [2]:


observation = channel_gain[3,:,:]
observation_ = observation
np.random.seed()  
for i in range(max_iteration):
  ru_limitation = 0
  user_limitation = 0
  user_unconform = 0
  ru_unconform = 0
  done = False
  evaluate = False
  
  action_pre = test_agent.choose_action(observation, evaluate) 
  action_pre = tf.reshape(action_pre,[numAPuser,numRU]).numpy()
  action_0 = np.zeros_like(action_pre)
  for k in range(numRU):  
    max_key = np.argmax(action_pre[:,k])
    action_0[max_key,k] = 1
  
  RU_mapper = np.stack((AP1_RU_mapper,AP2_RU_mapper,AP3_RU_mapper,action_0))

  for user in range(numAPuser):
    user_limitation = np.sum(action_0[user,:]==1)
    if user_limitation > 3:
      user_unconform += 1

 
  system_bitrate = test_env.calculate_4_cells(RU_mapper,linkmode)
  
  if user_unconform < 1:
    reward = 100*(1./(1 + np.exp(-system_bitrate))) 
  else:
    reward = -100*user_unconform
  print('iteration = ',i)
  print('pre_action',action_pre) 
  print('action',action_0)
  test_agent.remember(observation, action_0, reward, observation_, done)
    #print(observation)
    #print(action)
  test_agent.learn()
  critic_loss_history.append(test_agent.critic_loss_show)
  actor_loss_history.append(test_agent.actor_loss_show)
  system_bitrate_history.append(system_bitrate)
  reward_history.append(reward)
  #user_unconform_history.append(user_unconform)
  #ru_unconform_history.append(ru_unconform)
  #critic_value_history.append(test_agent.critic_value_show)
  #target_history.append(test_agent.target_show)
  print('critic_value',test_agent.critic_value_show)
  print('target',test_agent.target_show)
  print('ag',test_agent.ag_show)#ag_history.append()
  print('cg',test_agent.cg_show)
  #cg_history.append(test_agent.cg_show)
#print(len(critic_loss_history))
#print(len(critic_loss_history))
#print(len(reward_history)) 
  

NameError: name 'channel_gain' is not defined

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [91 lines of output]
      
      
      WARNING, No "Setup" File Exists, Running "buildconfig/config.py"
      Using WINDOWS configuration...
      
      Making dir :prebuilt_downloads:
      Downloading... https://www.libsdl.org/release/SDL2-devel-2.0.16-VC.zip 13d952c333f3c2ebe9b7bc0075b4ad2f784e7584
      Unzipping :prebuilt_downloads\SDL2-devel-2.0.16-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_image/release/SDL2_image-devel-2.0.5-VC.zip 137f86474691f4e12e76e07d58d5920c8d844d5b
      Unzipping :prebuilt_downloads\SDL2_image-devel-2.0.5-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_ttf/release/SDL2_ttf-devel-2.0.15-VC.zip 1436df41ebc47ac36e02ec9bda5699e80ff9bd27
      Unzipping :prebuilt_downloads\SDL2_ttf-devel-2.0.15-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_mixer/release/SDL2_mixer-devel-

  Obtaining dependency information for tf_agents[reverb] from https://files.pythonhosted.org/packages/0f/b0/d34b8986cbbccec4a989a0674e98453a3d4daa30059dce4345036aebe023/tf_agents-0.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for absl-py>=0.6.1 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/61.3 kB ? eta -:--:--
     ---------------------------------------- 61.3/61.3 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/624.4 kB ? eta -:--:--
     ------- ------------------------------ 122.9/624.4 kB 2.4 MB/s eta 0:00:01
     -------------- ----------------------- 245.8/624.4 kB 2.5 MB/s eta 0:00:01
     -------------------- ----------------- 337.9/624.4 kB 2.6 MB/s eta 0:00:01
     ---------------------------- --------- 471.0/624.4 kB 2.5 MB/s eta 0:00:01
     --------------------------------

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(reward_history)),reward_history,label="system bitrate")
plt.title('reward show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("reward.png")
files.download("reward.png") 
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(system_bitrate_history)),system_bitrate_history)
plt.title('system bitrate show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("system_bitrate.png")
files.download("system_bitrate.png") 
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(critic_loss_history)),critic_loss_history)
plt.title('critic loss show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("critic_loss.png")
files.download("critic_loss.png") 
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(actor_loss_history)),actor_loss_history)
plt.title('actor loss show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("actor_loss.png")
files.download("actor_loss.png") 
plt.show()

'''fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(user_unconform_history)),user_unconform_history)
plt.title('user unconform show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("user_unconform.png")
files.download("user_unconform.png") 
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(ru_unconform_history)),ru_unconform_history)
plt.title('ru unconform show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("ru_unconform.png")
files.download("ru_unconform.png") 
plt.show()'''
'''
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(target_history)),target_history)
plt.title('target show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("target.png")
files.download("target.png") 
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(ag_history)),ag_history)
plt.title('actor gradient show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("actor_gradient.png")
files.download("actor_gradient.png") 
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(range(len(cg_history)),cg_history)
plt.title('critic gradient show',fontsize=20)#标题，并设定字号大小
from google.colab import files
plt.savefig("critic_gradient.png")
files.download("critic_gradient.png") 
plt.show()'''

#print(critic_value_history)
